In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

2023-04-04 22:28:48.089360: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def load_dataset(dsname):
    metadata = pd.read_csv(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/metadata_final.csv")
    x = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_data0.p", "rb" ))
    y = pickle.load(open(f"/datasets/nicolas_facchinetti/processed_data/{dsname}/processed_labels.p", "rb" ))
    return metadata, x, y

In [3]:
from sklearn.preprocessing import StandardScaler

datasets = ["emodb", "emovo", "ravdess"]
scalers = {d: StandardScaler() for d in datasets}

data = {}
for d in datasets:
    md, x, y = load_dataset(d)
    data[d] = {}
    # standardize data
    n = x.shape
    data[d]["x"] = scalers[d].fit_transform(x.reshape(n[0],-1)).reshape(n)
    data[d]["y"] = y
    data[d]["metadata"] = md

In [4]:
from sklearn.model_selection import train_test_split

train = {}
test = {}
test_md = {}
for d in datasets:
    train[d] = {}
    test[d] = {}
    train_df, test_df = train_test_split(data[d]['metadata'], 
                                       test_size = 0.2, 
                                       random_state = 1938,
                                       stratify = data[d]['metadata']['label'])
    test_md[d] = test_df.reset_index()
    train_index = train_df.index
    test_index = test_df.index
    print(f'For {d}:\ttrain size {len(train_index)}, test size {len(test_index)}')
    train[d]['x'] = data[d]['x'][train_index]
    train[d]['y'] = data[d]['y'][train_index]
    test[d]['x']= data[d]['x'][test_index]
    test[d]['y']= data[d]['y'][test_index]

For emodb:	train size 3344, test size 837
For emovo:	train size 3944, test size 987
For ravdess:	train size 7161, test size 1791


In [5]:
def get_m_f_data(md):
    m = md[md.gender=='m'].index
    f = md[md.gender=='f'].index
    return m, f

test_mf = {}
for d in datasets:
    data_d = get_m_f_data(test_md[d])
    print(f"For {d}\n\tmale:{data_d[0].shape[0]}\n\tfemale:{data_d[1].shape[0]}\n")

For emodb
	male:315
	female:522

For emovo
	male:488
	female:499

For ravdess
	male:895
	female:896



In [6]:
models = {d: tf.keras.models.load_model(f'/datasets/nicolas_facchinetti/models/{d}')
          for d in datasets}

2023-04-04 22:28:57.271975: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-04 22:28:57.272859: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-04 22:28:57.378843: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:15:00.0 name: NVIDIA TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 607.97GiB/s
2023-04-04 22:28:57.378875: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-04-04 22:28:57.380479: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-04 22:28:57.380555: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2023-0

In [7]:
import keras_tuner as kt
param = pickle.load(open('/datasets/nicolas_facchinetti/param.p', "rb" ))

# ART Model

Find max and min value for each dataset to use as clip values for attacks (strictly needed for black box)

In [8]:
minv = np.array([data[d]['x'].min() for d in models])
minv = {d: v for v, d in zip(np.floor(minv), models)}

maxv = np.array([data[d]['x'].max() for d in models])
maxv = {d: v for v, d in zip(np.ceil(maxv), models)}
minv, maxv

({'emodb': -5.0, 'emovo': -3.0, 'ravdess': -2.0},
 {'emodb': 7.0, 'emovo': 7.0, 'ravdess': 8.0})

Make ART model

In [9]:
from art.estimators.classification import TensorFlowV2Classifier

def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

classifiers = {d: TensorFlowV2Classifier(models[d],
                                         nb_classes=5,
                                         loss_object=loss_object,
                                         train_step=train_step,
                                         input_shape=(261,128,1),
                                         clip_values=(minv[d], maxv[d]),
                                         channels_first=False)
               for d in models}

/home/nicolas/.conda/envs/thesis/lib/python3.9/site-packages/art/estimators/certification/__init__.py:14: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


# Adversarial training

In [10]:
from art.defences.trainer import AdversarialTrainer
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import DeepFool
from art.attacks.evasion import CarliniLInfMethod
from art.attacks.evasion import SaliencyMapMethod
from art.attacks.evasion import BoundaryAttack
from art.attacks.evasion import PixelAttack


adv_models = {}

def get_adv_model(d):
    conf = FastGradientMethod(estimator=classifiers[d],
                              eps=0.25,
                              batch_size=256)
    
    
    return AdversarialTrainer(classifiers[d], attacks=[conf], ratio=0.6)

In [11]:
adv_models['emodb'] = get_adv_model('emodb')
adv_models['emovo'] = get_adv_model('emovo')
adv_models['ravdess'] = get_adv_model('ravdess')

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    
new_models = {}

def evaluate(d, m):
    mi, fi = get_m_f_data(test_md[d])
    loss, accuracy = m.evaluate(test[d]['x'], test[d]['y'], verbose=0)
    print(f'\tAll data\ttest loss: {loss:.6f} accuracy: {accuracy:.6f}')
    lossm, accuracym = m.evaluate(test[d]['x'][mi], test[d]['y'][mi], verbose=0)
    print(f'\tMale data\ttest loss: {lossm:.6f} accuracy: {accuracym:.6f}')
    lossf, accuracyf = m.evaluate(test[d]['x'][fi], test[d]['y'][fi], verbose=0)
    print(f'\tFemale data\ttest loss: {lossf:.6f} accuracy: {accuracyf:.6f}\n')

for d in adv_models:
    print(f"\n\nTraining on {d}")
    
    evaluate(d, adv_models[d].get_classifier().get_params()['model'])
    
    adv_models[d].fit(train[d]['x'], train[d]['y'],
                      batch_size=128,
                      nb_epochs=30)
    new_models[d] = adv_models[d].get_classifier().get_params()['model']
    
    evaluate(d, new_models[d])



Training on emodb


2023-04-04 22:29:09.567282: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-04-04 22:29:09.568006: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3301490000 Hz
2023-04-04 22:29:09.992423: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-04-04 22:29:10.190753: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


	All data	test loss: 0.316978 accuracy: 0.909200
	Male data	test loss: 0.353135 accuracy: 0.895238
	Female data	test loss: 0.295158 accuracy: 0.917625



Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/30 [00:00<?, ?it/s]

...

Adversarial training epochs:   0%|          | 0/30 [00:00<?, ?it/s]

	All data	test loss: 0.317115 accuracy: 0.891591
	Male data	test loss: 0.284186 accuracy: 0.903689
	Female data	test loss: 0.349318 accuracy: 0.879759



Training on ravdess
	All data	test loss: 0.313163 accuracy: 0.910664
	Male data	test loss: 0.307363 accuracy: 0.910615
	Female data	test loss: 0.318956 accuracy: 0.910714



Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/30 [00:00<?, ?it/s]

	All data	test loss: 0.234990 accuracy: 0.921273
	Male data	test loss: 0.232488 accuracy: 0.922905
	Female data	test loss: 0.237490 accuracy: 0.919643



# Evaluate impact

Model attack function

In [13]:
import time

def do_attack(method, X):
    start = time.time()
    x_adv = method.generate(X, verbose=True)
    elapsed = time.time() - start
    print(f'Attack get {elapsed} seconds to complete')
    return x_adv, elapsed

In [14]:
def evaluate_attack(model, test_samples, test_metadata, x_adv):
    mi, fi = get_m_f_data(test_metadata)
    x = test_samples['x']
    y = test_samples['y']
    
    def evaluate_on_samples(model, adv, x, y):
        loss, accuracy = model.evaluate(adv, y, verbose=0)
        perturbation = np.mean(np.abs((adv - x)))
        print(f'\t\tloss: {loss:.5f}, accuracy: {accuracy:.5f}, avg perturbation: {perturbation:.5f}\n')
        return loss, accuracy, perturbation
    
    print('\tWhole test set')
    l1, a1, p1 = evaluate_on_samples(model, x_adv, x, y)
    print('\tOnly male data')
    l2, a2, p2 = evaluate_on_samples(model, x_adv[mi], x[mi], y[mi])
    print('\tOnly female data')
    l3, a3, p3 = evaluate_on_samples(model, x_adv[fi], x[fi], y[fi])
    
    def make_dict(v):
        return {'All': v[0], 'Male': v[1], 'Female': v[2]}
    
    res_dict = {'Loss': make_dict([l1,l2,l3]),
                'Accuracy': make_dict([a1,a2,a3]),
                'Perturbation': make_dict([p1,p2,p3])}
    return res_dict

In [15]:
def attack(attacks_confs, name):
    # save results for dataframe output
    results[name] = {}
    times[name] = {}
    
    # return to store adv generated
    adv_generated = {}
    adv_generated[name] = {}
    for d in attacks_confs:
        results[name][d] = {}
        times[name][d] = {}
        adv_generated[name][d] = {}
        for p in attacks_confs[d]:
            print(f'Attacking {d} with {name}, parameter {p}')
            
            # compute adv examples and get time
            x_adv, time = do_attack(attacks_confs[d][p], test[d]['x'])
            # test adv examples on whole data, only M, and only F
            res = evaluate_attack(new_models[d], test[d], test_md[d], x_adv)
            
            results[name][d][p] = res
            times[name][d][p] = time
            adv_generated[name][d][p] = x_adv
            
        print("_____________________________")
    return adv_generated

Functions to display data in DataFrame

In [16]:
def df_acc(d_res):
    reform = {(dataset, param): values
              for dataset, inner1 in d_res.items()
              for param, inner2 in inner1.items()
              for metric, values in inner2.items()
              if metric == 'Accuracy'}
    mux = pd.MultiIndex.from_tuples(reform.keys())
    df = pd.DataFrame(list(reform.values()), index=mux)
    return df.unstack(level=1).stack(level=0)

In [17]:
def save_adv(data):
    for attack in data:
        pickle.dump(data[attack],
                    open(f'/datasets/nicolas_facchinetti/adv2/{attack}.p', "wb"))
    

In [18]:
# size of the batch on which adversarial samples are generated.
bs = 256
# store accuracy, loss and perturbation levels
results = {}
# store time to generate the samples
times = {}

Various attacks

In [19]:
epss = [0.25, 0.5, 0.75, 1, 1.25]
conf1 = {d: {e: FastGradientMethod(estimator=classifiers[d],
                                   eps=e,
                                   batch_size=bs)
             for e in epss}
         for d in datasets}

adv = attack(conf1, 'fgsm')

Attacking emodb with fgsm, parameter 0.25
Attack get 1.7614452838897705 seconds to complete
	Whole test set
		loss: 1.05391, accuracy: 0.53524, avg perturbation: 0.21857

	Only male data
		loss: 1.09779, accuracy: 0.50794, avg perturbation: 0.21847

	Only female data
		loss: 1.02743, accuracy: 0.55172, avg perturbation: 0.21862

Attacking emodb with fgsm, parameter 0.5
Attack get 1.2305073738098145 seconds to complete
	Whole test set
		loss: 1.33379, accuracy: 0.41577, avg perturbation: 0.43713

	Only male data
		loss: 1.33439, accuracy: 0.38095, avg perturbation: 0.43694

	Only female data
		loss: 1.33342, accuracy: 0.43678, avg perturbation: 0.43725

Attacking emodb with fgsm, parameter 0.75
Attack get 1.1699957847595215 seconds to complete
	Whole test set
		loss: 1.59393, accuracy: 0.31063, avg perturbation: 0.65570

	Only male data
		loss: 1.55714, accuracy: 0.29841, avg perturbation: 0.65540

	Only female data
		loss: 1.61612, accuracy: 0.31801, avg perturbation: 0.65587

Attackin

In [20]:
df_acc(results['fgsm'])

0.25      0.50      0.75      1.00      1.25
emodb   All     0.535245  0.415771  0.310633  0.258065  0.231780
        Female  0.551724  0.436782  0.318008  0.270115  0.245211
        Male    0.507937  0.380952  0.298413  0.238095  0.209524
emovo   All     0.742655  0.286727  0.257345  0.234043  0.226950
        Female  0.751503  0.264529  0.254509  0.244489  0.230461
        Male    0.733607  0.309426  0.260246  0.223361  0.223361
ravdess All     0.610274  0.364601  0.296482  0.280290  0.254606
        Female  0.629464  0.369420  0.310268  0.279018  0.250000
        Male    0.591061  0.359777  0.282682  0.281564  0.259218

In [21]:
epss = [0.25]
conf2 = {d: {e: BasicIterativeMethod(estimator=classifiers[d],
                                     eps=e,
                                     eps_step=0.1,
                                     batch_size=bs,
                                     max_iter=100,
                                     verbose=True)
             for e in epss}
         for d in datasets}

adv = attack(conf2, 'bim')

Attacking emodb with bim, parameter 0.25


PGD - Batches: 0it [00:00, ?it/s]

Attack get 24.66125249862671 seconds to complete
	Whole test set
		loss: 9.24198, accuracy: 0.05854, avg perturbation: 0.16014

	Only male data
		loss: 9.38804, accuracy: 0.06349, avg perturbation: 0.16028

	Only female data
		loss: 9.15384, accuracy: 0.05556, avg perturbation: 0.16006

_____________________________
Attacking emovo with bim, parameter 0.25


PGD - Batches: 0it [00:00, ?it/s]

Attack get 28.59143352508545 seconds to complete
	Whole test set
		loss: 19.83698, accuracy: 0.07396, avg perturbation: 0.15013

	Only male data
		loss: 20.26051, accuracy: 0.05943, avg perturbation: 0.14957

	Only female data
		loss: 19.42279, accuracy: 0.08818, avg perturbation: 0.15067

_____________________________
Attacking ravdess with bim, parameter 0.25


PGD - Batches: 0it [00:00, ?it/s]

Attack get 51.4910089969635 seconds to complete
	Whole test set
		loss: 25.10628, accuracy: 0.05918, avg perturbation: 0.14723

	Only male data
		loss: 25.65358, accuracy: 0.05922, avg perturbation: 0.14582

	Only female data
		loss: 24.55960, accuracy: 0.05915, avg perturbation: 0.14864

_____________________________


In [22]:
df_acc(results['bim'])

0.25
emodb   All     0.058542
        Male    0.063492
        Female  0.055556
emovo   All     0.073961
        Male    0.059426
        Female  0.088176
ravdess All     0.059185
        Male    0.059218
        Female  0.059152

In [23]:
iterations = [5]
conf3 = {d: {i: DeepFool(classifier=classifiers[d],
                         nb_grads=5,
                         epsilon=1e-05,  # to help convergence and avoid being stuck to 0
                         max_iter=i,
                         batch_size=bs,
                         verbose=True)
             for i in iterations}
         for d in datasets}

adv = attack(conf3, 'df')

Attacking emodb with df, parameter 5


DeepFool:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 10.00229001045227 seconds to complete
	Whole test set
		loss: 2.46775, accuracy: 0.05854, avg perturbation: 1.15039

	Only male data
		loss: 2.47233, accuracy: 0.06032, avg perturbation: 0.95508

	Only female data
		loss: 2.46498, accuracy: 0.05747, avg perturbation: 1.26824

_____________________________
Attacking emovo with df, parameter 5


DeepFool:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 17.130547285079956 seconds to complete
	Whole test set
		loss: 2.65238, accuracy: 0.06586, avg perturbation: 0.74321

	Only male data
		loss: 2.64622, accuracy: 0.04918, avg perturbation: 0.74186

	Only female data
		loss: 2.65840, accuracy: 0.08216, avg perturbation: 0.74454

_____________________________
Attacking ravdess with df, parameter 5


DeepFool:   0%|          | 0/7 [00:00<?, ?it/s]

Attack get 23.44492197036743 seconds to complete
	Whole test set
		loss: 2.93182, accuracy: 0.05639, avg perturbation: 1.27718

	Only male data
		loss: 3.04556, accuracy: 0.05810, avg perturbation: 1.15520

	Only female data
		loss: 2.81820, accuracy: 0.05469, avg perturbation: 1.39902

_____________________________


In [24]:
df_acc(results['df'])

5
emodb   All     0.058542
        Male    0.060317
        Female  0.057471
emovo   All     0.065856
        Male    0.049180
        Female  0.082164
ravdess All     0.056393
        Male    0.058101
        Female  0.054688

In [25]:
# amount of pertubration
thetas = [1]
conf4 = {d: {t: SaliencyMapMethod(classifier=classifiers[d],
                                  theta=t, # amount of perturbation introduced per step
                                  gamma=1, #  maximum fraction of features being perturbed
                                  batch_size=bs,
                                  verbose=True)                           
             for t in thetas}
         for d in datasets}

adv = attack(conf4, 'jsma')

Attacking emodb with jsma, parameter 1


JSMA:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 485.83380603790283 seconds to complete
	Whole test set
		loss: 1.23493, accuracy: 0.01314, avg perturbation: 0.00590

	Only male data
		loss: 1.29309, accuracy: 0.01587, avg perturbation: 0.00488

	Only female data
		loss: 1.19984, accuracy: 0.01149, avg perturbation: 0.00651

_____________________________
Attacking emovo with jsma, parameter 1


JSMA:   0%|          | 0/4 [00:00<?, ?it/s]

Attack get 309.6410312652588 seconds to complete
	Whole test set
		loss: 1.13276, accuracy: 0.01520, avg perturbation: 0.00337

	Only male data
		loss: 1.11844, accuracy: 0.00615, avg perturbation: 0.00361

	Only female data
		loss: 1.14677, accuracy: 0.02405, avg perturbation: 0.00314

_____________________________
Attacking ravdess with jsma, parameter 1


JSMA:   0%|          | 0/7 [00:00<?, ?it/s]

Attack get 621.0338711738586 seconds to complete
	Whole test set
		loss: 1.14391, accuracy: 0.02066, avg perturbation: 0.00323

	Only male data
		loss: 1.16344, accuracy: 0.01788, avg perturbation: 0.00246

	Only female data
		loss: 1.12440, accuracy: 0.02344, avg perturbation: 0.00399

_____________________________


In [26]:
df_acc(results['jsma'])

1
emodb   All     0.013142
        Male    0.015873
        Female  0.011494
emovo   All     0.015198
        Male    0.006148
        Female  0.024048
ravdess All     0.020659
        Male    0.017877
        Female  0.023438

In [27]:
cs = [1e-04]
conf7 = {d: {c: CarliniLInfMethod(classifier=classifiers[d],
                                  max_iter=10,
                                  learning_rate=0.01,
                                  initial_const=c,
                                  largest_const=2e01,
                                  decrease_factor=0.8,
                                  const_factor=4,
                                  batch_size=bs,
                                  verbose=True)
             for c in cs}
         for d in datasets}

adv = attack(conf7, 'cwinf')

Attacking emodb with cwinf, parameter 0.0001


C&W L_inf:   0%|          | 0/837 [00:00<?, ?it/s]

Attack get 4247.645616531372 seconds to complete
	Whole test set
		loss: 1.61663, accuracy: 0.06093, avg perturbation: 0.06300

	Only male data
		loss: 1.63000, accuracy: 0.06032, avg perturbation: 0.06003

	Only female data
		loss: 1.60856, accuracy: 0.06130, avg perturbation: 0.06480

_____________________________
Attacking emovo with cwinf, parameter 0.0001


C&W L_inf:   0%|          | 0/987 [00:00<?, ?it/s]

Attack get 5023.99459862709 seconds to complete
	Whole test set
		loss: 2.72636, accuracy: 0.07194, avg perturbation: 0.03641

	Only male data
		loss: 2.82651, accuracy: 0.05533, avg perturbation: 0.03602

	Only female data
		loss: 2.62841, accuracy: 0.08818, avg perturbation: 0.03679

_____________________________
Attacking ravdess with cwinf, parameter 0.0001


C&W L_inf:   0%|          | 0/1791 [00:00<?, ?it/s]

Attack get 10139.293450832367 seconds to complete
	Whole test set
		loss: 2.94857, accuracy: 0.05918, avg perturbation: 0.02960

	Only male data
		loss: 3.01784, accuracy: 0.06145, avg perturbation: 0.02801

	Only female data
		loss: 2.87938, accuracy: 0.05692, avg perturbation: 0.03118

_____________________________


In [28]:
df_acc(results['cwinf'])

0.0001
emodb   All     0.060932
        Male    0.060317
        Female  0.061303
emovo   All     0.071935
        Male    0.055328
        Female  0.088176
ravdess All     0.059185
        Male    0.061453
        Female  0.056920

In [29]:
# amount of pertubration
thetas = [10]
conf8 = {d: {t: PixelAttack(classifier=classifiers[d],
                            max_iter=20, # max iter evolution strategy
                            es=1,    # DE as CMA-ES not aviable
                            th=t,
                            verbose=True)                           
             for t in thetas}
         for d in datasets}

adv = attack(conf8, 'pa')

Attacking emodb with pa, parameter 10


Pixel threshold: 0it [00:00, ?it/s]

Attack get 2812.707174062729 seconds to complete
	Whole test set
		loss: 0.49482, accuracy: 0.68698, avg perturbation: 0.00038

	Only male data
		loss: 0.57104, accuracy: 0.67302, avg perturbation: 0.00039

	Only female data
		loss: 0.44883, accuracy: 0.69540, avg perturbation: 0.00037

_____________________________
Attacking emovo with pa, parameter 10


Pixel threshold: 0it [00:00, ?it/s]

Attack get 2004.5466578006744 seconds to complete
	Whole test set
		loss: 1.24121, accuracy: 0.39716, avg perturbation: 0.00076

	Only male data
		loss: 1.19581, accuracy: 0.42008, avg perturbation: 0.00075

	Only female data
		loss: 1.28562, accuracy: 0.37475, avg perturbation: 0.00077

_____________________________
Attacking ravdess with pa, parameter 10


Pixel threshold: 0it [00:00, ?it/s]

Attack get 3482.7025468349457 seconds to complete
	Whole test set
		loss: 2.10095, accuracy: 0.37186, avg perturbation: 0.00088

	Only male data
		loss: 2.76071, accuracy: 0.26369, avg perturbation: 0.00102

	Only female data
		loss: 1.44192, accuracy: 0.47991, avg perturbation: 0.00073

_____________________________


In [30]:
df_acc(results['pa'])

10
emodb   All     0.686977
        Male    0.673016
        Female  0.695402
emovo   All     0.397163
        Male    0.420082
        Female  0.374750
ravdess All     0.371859
        Male    0.263687
        Female  0.479911

In [31]:
# amount of pertubration
eps = [5]
conf9 = {d: {e: BoundaryAttack(estimator=classifiers[d],
                               delta=0.01,     # dynamic adjusted
                               epsilon=0.01,   # dynamic adjusted
                               step_adapt=0.5, # suggested in the paper
                               max_iter=100,
                               num_trial=3,
                               sample_size=10,
                               targeted=False,
                               batch_size=bs,
                               verbose=True)                       
             for e in eps}
         for d in datasets}
    
adv = attack(conf9, 'ba')

Attacking emodb with ba, parameter 5


Boundary attack:   0%|          | 0/837 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

...

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Boundary attack - iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Attack get 2242.280050754547 seconds to complete
	Whole test set
		loss: 1.44602, accuracy: 0.14629, avg perturbation: 1.37335

	Only male data
		loss: 1.45422, accuracy: 0.14190, avg perturbation: 1.55656

	Only female data
		loss: 1.43783, accuracy: 0.15067, avg perturbation: 1.19035

_____________________________


In [32]:
df_acc(results['ba'])

5
emodb   All     0.056153
        Male    0.053968
        Female  0.057471
emovo   All     0.209726
        Male    0.192623
        Female  0.226453
ravdess All     0.146287
        Male    0.141899
        Female  0.150670